In [1]:
!pip install kaggle
!pip install librosa
!pip install torch
!pip install scikit-learn
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

import kagglehub

# Download latest version
path = kagglehub.dataset_download("itachi9604/disease-symptom-description-dataset")

print("Path to dataset files:", path)

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


100%|██████████| 30.1k/30.1k [00:00<00:00, 8.37MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/itachi9604/disease-symptom-description-dataset/versions/2


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix,classification_report,confusion_matrix,precision_score,roc_curve
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import warnings
import torch
import os

In [3]:
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

DATASET_PATH = path

class_dir = os.path.join(DATASET_PATH)
for filename in os.listdir(class_dir):
  print(filename)

Using device: cuda
Symptom-severity.csv
symptom_precaution.csv
symptom_Description.csv
dataset.csv


**Read and shuffle the dataset**

In [4]:
df = pd.read_csv(os.path.join(DATASET_PATH, 'dataset.csv'))
df = shuffle(df,random_state=42)

for col in df.columns:
    df[col] = df[col].str.replace('_',' ')

null_checker = df.apply(lambda x: sum(x.isnull())).to_frame(name='count')

cols = df.columns
data = df[cols].values.flatten()
s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)

df = df.fillna(0)

df1 = pd.read_csv(os.path.join(DATASET_PATH, 'Symptom-severity.csv'))
x=df1['Symptom']

dfx=pd.DataFrame()
dfx["Disease"]=df["Disease"]
y=0
dfx[x]=0
for index, row in df.iterrows():
    for symptom in df.columns[1:]:
        if row[symptom] != 0:
            dfx.loc[index, row[symptom]] = 1
dfx = dfx.fillna(0)
dfx[dfx.columns[1:]]=dfx[dfx.columns[1:]].astype('int')
dfx.columns = dfx.columns.str.strip()

symptom_sums = dfx.iloc[:, 1:].sum(axis=0)
symptoms_with_no_values = symptom_sums[symptom_sums == 0].index.tolist()

columns_to_drop = symptoms_with_no_values
dfx = dfx.drop(columns=columns_to_drop)
dfx[dfx.columns[1:]].sum(axis=0).sort_values()
print(dfx.columns.to_list())
y=df['Disease'].unique()

data = dfx.iloc[:,1:].values
labels = dfx['Disease'].values

['Disease', 'itching', 'shivering', 'chills', 'acidity', 'vomiting', 'fatigue', 'anxiety', 'restlessness', 'lethargy', 'cough', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'nausea', 'constipation', 'diarrhoea', 'malaise', 'phlegm', 'congestion', 'dizziness', 'cramps', 'bruising', 'obesity', 'unsteadiness', 'depression', 'irritability', 'polyuria', 'coma', 'palpitations', 'blackheads', 'scurring', 'blister', 'skin rash', 'pus filled pimples', 'mood swings', 'weight loss', 'fast heart rate', 'excessive hunger', 'muscle weakness', 'abnormal menstruation', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts', 'yellowish skin', 'loss of appetite', 'abdominal pain', 'yellowing of eyes', 'chest pain', 'loss of balance', 'lack of concentration', 'blurred and distorted vision', 'drying and tingling lips', 'slurred speech', 'stiff neck', 'swelling joints', 'painful walking', 'dark urine', 'yellow urine', 'receiving blood transfusion', 'receivi

In [5]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.8,random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3936, 131) (984, 131) (3936,) (984,)


In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y=le.classes_
y

array(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes',
       'Dimorphic hemmorhoids(piles)', 'Drug Reaction',
       'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Hypertension', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins',
       'hepatitis A'], dtype=object)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score
import pickle
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=kfold, scoring='f1_weighted', verbose=1)
grid_search.fit(x_train, y_train)
print(f'Best parameters: {grid_search.best_params_}')
model = grid_search.best_estimator_
cv_results = pd.DataFrame(grid_search.cv_results_)


Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best parameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [8]:
cv_results = pd.DataFrame(grid_search.cv_results_)
# print(cv_results.head())

test_predictions = model.predict(x_test)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")
test_f1 = f1_score(y_test, test_predictions, average='weighted')
test_roc = roc_auc_score(y_test, model.predict_proba(x_test), multi_class='ovr')
print(f'RandomForest test F1 Score: {test_f1:.4f}, AUC-ROC Score: {test_roc:.4f}')


with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("Best model saved as 'model.pkl'")
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(le, le_file)

Test Accuracy: 1.0000
RandomForest test F1 Score: 1.0000, AUC-ROC Score: 1.0000
Best model saved as 'model.pkl'


In [16]:
from sklearn.preprocessing import LabelEncoder

def preprocess_and_predict(user_input, model, label_encoder, symptom_columns):
  with open(model, 'rb') as f:
      model = pickle.load(f)

  with open(label_encoder, 'rb') as le_file:
    le = pickle.load(le_file)

  user_df = pd.DataFrame([user_input], columns=symptom_columns)
  user_df.fillna(0, inplace=True)
  input_data = user_df.values
  prediction = model.predict(input_data)
  probabilities = model.predict_proba(input_data)
  predicted_label = le.inverse_transform(prediction)
  prob_dict = {le.inverse_transform([i])[0]: prob for i, prob in enumerate(probabilities[0])}
  return predicted_label[0], prob_dict

symptom_columns = ['itching', 'shivering', 'chills', 'acidity', 'vomiting', 'fatigue', 'anxiety', 'restlessness', 'lethargy', 'cough', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'nausea', 'constipation', 'diarrhoea', 'malaise', 'phlegm', 'congestion', 'dizziness', 'cramps', 'bruising', 'obesity', 'unsteadiness', 'depression', 'irritability', 'polyuria', 'coma', 'palpitations', 'blackheads', 'scurring', 'blister', 'skin rash', 'pus filled pimples', 'mood swings', 'weight loss', 'fast heart rate', 'excessive hunger', 'muscle weakness', 'abnormal menstruation', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts', 'yellowish skin', 'loss of appetite', 'abdominal pain', 'yellowing of eyes', 'chest pain', 'loss of balance', 'lack of concentration', 'blurred and distorted vision', 'drying and tingling lips', 'slurred speech', 'stiff neck', 'swelling joints', 'painful walking', 'dark urine', 'yellow urine', 'receiving blood transfusion', 'receiving unsterile injections', 'visual disturbances', 'burning micturition', 'bladder discomfort', 'foul smell of urine', 'continuous feel of urine', 'irregular sugar level', 'increased appetite', 'joint pain', 'skin peeling', 'small dents in nails', 'inflammatory nails', 'swelling of stomach', 'distention of abdomen', 'history of alcohol consumption', 'fluid overload', 'pain during bowel movements', 'pain in anal region', 'bloody stool', 'irritation in anus', 'acute liver failure', 'stomach bleeding', 'back pain', 'weakness in limbs', 'neck pain', 'mucoid sputum', 'mild fever', 'muscle pain', 'family history', 'continuous sneezing', 'watering from eyes', 'rusty sputum', 'weight gain', 'puffy face and eyes', 'enlarged thyroid', 'brittle nails', 'swollen extremeties', 'swollen legs', 'prominent veins on calf', 'stomach pain', 'spinning movements', 'sunken eyes', 'silver like dusting', 'swelled lymph nodes', 'blood in sputum', 'swollen blood vessels', 'toxic look (typhos)', 'belly pain', 'throat irritation', 'redness of eyes', 'sinus pressure', 'runny nose', 'loss of smell', 'passage of gases', 'cold hands and feets', 'weakness of one body side', 'altered sensorium', 'nodal skin eruptions', 'red sore around nose', 'yellow crust ooze', 'ulcers on tongue', 'spotting  urination', 'pain behind the eyes', 'red spots over body', 'internal itching', 'movement stiffness', 'knee pain', 'hip joint pain', 'dischromic  patches']

user_input = {
    'itching': 1,
    'skin_rash': 1,
    'nodal_skin_eruptions': 0,
    'continuous_sneezing': 1,
    'shivering': 1,
    'chills': 1,
    'joint_pain': 1,
    'stomach_pain': 1,
}

# Predict the disease
predicted_disease, probability = preprocess_and_predict(user_input, 'model.pkl', 'label_encoder.pkl', symptom_columns)
print(f"Predicted Disease: {predicted_disease}")
print(f"Probability : {probability}")

Predicted Disease: Allergy
Probability : {'(vertigo) Paroymsal  Positional Vertigo': 0.02097418387840458, 'AIDS': 0.023751005562590476, 'Acne': 0.0289661776554952, 'Alcoholic hepatitis': 0.024234317008768616, 'Allergy': 0.13190607713185248, 'Arthritis': 0.024412689611574792, 'Bronchial Asthma': 0.023834230257432112, 'Cervical spondylosis': 0.026916449966862255, 'Chicken pox': 0.022708766406238614, 'Chronic cholestasis': 0.03333063909401234, 'Common Cold': 0.010394636021880272, 'Dengue': 0.01351321901627071, 'Diabetes': 0.010674954156752132, 'Dimorphic hemmorhoids(piles)': 0.02343848724217687, 'Drug Reaction': 0.03212244076920094, 'Fungal infection': 0.030416128357880035, 'GERD': 0.025500362072155586, 'Gastroenteritis': 0.025245629271429842, 'Heart attack': 0.02959935671396796, 'Hepatitis B': 0.016050191559708958, 'Hepatitis C': 0.025901502216827802, 'Hepatitis D': 0.025446476996172102, 'Hepatitis E': 0.017403559388233313, 'Hypertension': 0.02565808106791214, 'Hyperthyroidism': 0.013725